## set up telegram notifications

не очень понятно, нужно ли это. если нужно -- напишите @oserikov в телеграме, я расскажу, что сделать, чтобы присылались сообщения с качеством модели когда она отработает. 

In [0]:
telegram_notifications_enabled=False
EXP_DESCRIPTION = "WORD LEVEL INPUT EMBEDDINGS"

In [0]:
if telegram_notifications_enabled:
    bot_token = input("введите telegram bot token: ")
    chat_id = "292749902" # for @oserikov

# prepare

In [0]:
import sys

#### install prereqs

In [0]:

# clone openmt-py used in calma and move it into the proper folder
!git clone https://github.com/OpenNMT/OpenNMT-py.git
%cd OpenNMT-py
!git checkout -b stable-version d57fa68e6b0c2041642af40f76e1d5903c80a9b8
%cd ..
!mv OpenNMT-py ~
!wget -q https://raw.githubusercontent.com/NIS-2018-CROSS-M/calma/tmp-utils/utils/onmt-decoder.py -O ~/OpenNMT-py/onmt/decoders/decoder.py
!wget -q https://raw.githubusercontent.com/NIS-2018-CROSS-M/calma/tmp-utils/utils/onmt-opts.py -O ~/OpenNMT-py/onmt/opts.py

# clone and run a tool installing pytorch 0.4.1 with cuda 9.2 into colab (maybe works on any ubuntu 16)
!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git colab_cuda_upgrader
!bash colab_cuda_upgrader/pytorch041_cuda92_colab.sh


# install dependencies used in calma project
!{sys.executable} -m pip install configargparse

# install the proper version of torchtext
!git clone https://github.com/pytorch/text.git
%cd text
!{sys.executable} -m pip install .
%cd ..

# receive the calma
!git clone https://github.com/ftyers/calma.git
%cd calma
!git checkout -b latest-known-version d4ce3758d06538933855f734a44630efc8e2b6b2
%cd sharedtaskdata
!rm onmt-data/*
!rm results/*

#### imports

In [0]:
from collections import defaultdict as dd
from random import shuffle
import re
import urllib

## helping functions definitions

In [0]:
def initialize_data(train_src, train_tgt, valid_src, valid_tgt, prepared_training_data_prefix):
    prepr_params = f"-train_src {train_src} -train_tgt {train_tgt} -valid_src {valid_src} -valid_tgt {valid_tgt} -save_data {prepared_training_data_prefix}" 
    !{sys.executable} ~/OpenNMT-py/preprocess.py {prepr_params}
    
    
def train_ml(train_params):
    train_params = " ".join(train_params)
    !{sys.executable} ~/OpenNMT-py/train.py {train_params}

                
def generate_predictions(generation_params, output_filename):
    generation_params = " ".join(generation_params)
    !{sys.executable} ~/OpenNMT-py/translate.py {generation_params} > {output_filename}

    
def choose_best_predictions(nbest_filename, covered_filename, output_filename):
    !cat {nbest_filename} | grep -v -P "^\s+" | grep -v -P "^\+" | {sys.executable} scripts/get-analyses.py 0.8 3 {covered_filename} > {output_filename}


def score_predictions(chosen_output_filename, uncovered_filename, score_output_filename):
    !{sys.executable} scripts/eval_tabular.py {chosen_output_filename} {uncovered_filename} \
     >> {score_output_filename}


In [0]:
def modify_nbest(nbest_src_filename, nbest_tgt_filename, nbestModifyer):
    with open(nbest_src_filename, 'r', encoding='utf-8') as src_f, \
        open(nbest_tgt_filename, 'w', encoding='utf-8') as tgt_f:

        for line in src_f.readlines():
            line = line.rstrip('\n').rstrip('\r')
            if line.startswith("SENT "):
                line = nbestModifyer.sent_to_baseline_compatible(line)
            elif re.match("^\[[\-\+]?\d+\.\d+\]\s\[", line):
                line = nbestModifyer.hyp_to_baseline_compatible(line)

            print(line, file=tgt_f)

#### def data_generation

In [0]:
# the method called for each non-processed training data row
def get_data_entry(language, wordform, lemma, pos_tag, morphological_analysis):
    lemma = ' '.join(lemma)
    wordform = ' '.join(wordform)
    morphological_analysis = morphological_analysis.split('|')
    return wordform, '%s %s' % (lemma, ' '.join(['+%s' % x for x in [pos_tag] + morphological_analysis  + ["Language=%s" % language]]))


def generate_onmt_data(fn, res_src_fn, res_tgt_fn, DataModifyerClass):
    
    modify_src_line = DataModifyerClass.modify_src_line
    modify_tgt_line = DataModifyerClass.modify_tgt_line
    restore_orig_src_line = DataModifyerClass.restore_src_line
    restore_orig_tgt_line = DataModifyerClass.restore_tgt_line
    
    analyses = dd(set)

    for line in open(fn, encoding='utf-8'):
        line = line.rstrip('\n').rstrip('\r')
        lang, wf, lemma, pos, msd = line.split('\t')
        wf, a = get_data_entry(lang, wf, lemma, pos, msd)
        analyses[wf].add(a)
    
    tmp_src_fn = res_src_fn + "-default"
    tmp_tgt_fn = res_tgt_fn + "-default"
    
    tmp_src = open(tmp_src_fn, 'w')
    tmp_tgt = open(tmp_tgt_fn, 'w')
    res_src = open(res_src_fn, 'w')
    res_tgt = open(res_tgt_fn, 'w')
    
    analyses = list(analyses.items())
    shuffle(analyses)

    for wf, analysis in analyses:
        for a in analysis:
            print(wf, file = tmp_src)
            print(a, file = tmp_tgt)
            assert restore_orig_src_line(modify_src_line(wf)) == wf
            assert restore_orig_tgt_line(modify_tgt_line(a)) == a
            print(modify_src_line(wf), file = res_src)
            print(modify_tgt_line(a), file = res_tgt)


### def ml()

In [0]:
def ml(langs, tracks, train_params, dataModifyer, nbestModifyer):
    
    def generate_data(orig_data_fn, res_src_fn, res_tgt_fn):
        return generate_onmt_data(orig_data_fn, res_src_fn, res_tgt_fn, dataModifyer)

    def train(train_res_src_fn, train_res_tgt_fn, val_res_src_fn, val_res_tgt_fn, save_model_fn, train_params):
        data_fn = save_model_fn + "-prepared_training_data" #f"onmt-data/{lang}-track{track}"    
        initialize_data(train_res_src_fn, train_res_tgt_fn, val_res_src_fn, val_res_tgt_fn, data_fn)

        train_params.extend([f"-data {data_fn}", f"-save_model {save_model_fn}"])
        train_ml(train_params)
        !mv {save_model_fn}_step_{train_steps}.pt {save_model_fn}

    
    def predict(model_filename, input_data_filename, covered_filename, chosen_output_filename):
        output_data_filename = f"{input_data_filename}.out"
        nbest_output_filename = f"{input_data_filename}.nbest.out"
        prediction_params = [
            f"-model {model_filename}",
            f"-src {input_data_filename}",
            f"-output {output_data_filename}",
            f"-replace_unk",
            f"-verbose",
            f"-n_best 10",
            f"-beam 10"
        ]

        generate_predictions(prediction_params, nbest_output_filename)
        nbest_output_modified_filename = nbest_output_filename+"-modified"
        modify_nbest(nbest_output_filename, nbest_output_modified_filename, nbestModifyer)
        choose_best_predictions(nbest_output_modified_filename, covered_filename, chosen_output_filename)
    
    for lang in langs:
        for track in tracks:
            train_covered_filename = f"train/{lang}-track{track}-covered"
            train_uncovered_filename = f"train/{lang}-track{track}-uncovered"
            val_covered_filename = f"dev/{lang}-covered"
            val_uncovered_filename = f"dev/{lang}-uncovered"
            test_covered_filename = f"test/{lang}-covered"
            test_uncovered_filename = f"test/{lang}-uncovered"

            train_res_src_filename = f"onmt-data/{lang}-track{track}-src-train.txt"
            train_res_tgt_filename = f"onmt-data/{lang}-track{track}-tgt-train.txt"

            val_res_src_filename = f"onmt-data/{lang}-track{track}-src-dev.txt"
            val_res_tgt_filename = f"onmt-data/{lang}-track{track}-tgt-dev.txt"

            test_res_src_filename = f"onmt-data/{lang}-track{track}-src-test.txt"
            test_res_tgt_filename = f"onmt-data/{lang}-track{track}-tgt-test.txt"


            generate_data(train_uncovered_filename, train_res_src_filename, train_res_tgt_filename)        
            generate_data(val_uncovered_filename, val_res_src_filename, val_res_tgt_filename)

            model_filename = f"models/{lang}-track{track}.model"
            train(train_res_src_filename, train_res_tgt_filename, val_res_src_filename, val_res_tgt_filename, model_filename, train_params)


            score_log_filename = f"/content/calma/sharedtaskdata/{lang}-{track}-score.log"
            !echo "" > {score_log_filename}

            generate_data(test_covered_filename, test_res_src_filename, test_res_tgt_filename)
            test_pred_output_filename = f"results/{lang}-track{track}-test-covered.sys"
            predict(model_filename, test_res_src_filename, test_covered_filename, test_pred_output_filename)
            !echo "*===QUALITY ON TEST DATA===*" >> {score_log_filename}
            score_predictions(test_pred_output_filename, test_uncovered_filename, score_log_filename)

            val_pred_output_filename = f"results/{lang}-track{track}-dev-covered.sys"
            predict(model_filename, val_res_src_filename, val_covered_filename, val_pred_output_filename)
            !echo "*===QUALITY ON VAL DATA===*" >> {score_log_filename}
            score_predictions(val_pred_output_filename, val_uncovered_filename, score_log_filename)

            !cat {score_log_filename}
            
            if telegram_notifications_enabled:
                # в строку telegram_message записывается сообщение, которое будет отправлено в телеграме. 
                # сейчас это только тег #score и текст файла, содержащего отчет о качестве.
                # чтобы было легче жить, стоит дописать в эту строку описание эксперимента.
                telegram_message = "#score\n"+''.join(open(score_log_filename).readlines())+'\n'+EXP_DESCRIPTION

                telegram_message_encoded = urllib.parse.quote(telegram_message)
                !curl -i -X GET "https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={telegram_message_encoded}&parse_mode=markdown"


# ML

## set ml params

In [0]:
langs=['ast']
tracks=['1']
data_classes = ['test', 'dev']

train_steps=1000
valid_steps=100
save_checkpoint_steps = valid_steps

train_params = [
    f"-train_steps {train_steps}",
    f"-valid_steps {valid_steps}",
    f"-save_checkpoint_steps {save_checkpoint_steps}",
    f"-world_size 1",
    f"-gpu_ranks 0 1",
    f"-encoder_type brnn"
]

## Word-level embeddings on input approach

#### data description

**source**
```
wf1
```

**target**
```
l1 l2 ... lN +POS +Tag1=Value1 ... +TagN=ValueN +Language=langCode
```

**uncovered**
(tab separated)
```
langCode	wordForm	lemma	POS	Tag1=Value1|...|TagN=ValueN
```

**prediction** raw
```
SENT 1: ['wf1']
...
[-9.2825] ['c', 'o', 'n', 'v', 'i', 'd', 'u', '+NOUN', '+Gender=Masc', '+Number=Plur', '+Language=ast']
```
**prediction** passed to `eval()`
```
['l1', 'l2', ..., 'lN', '+POS', '+Tag1=Value1', ..., '+TagN=ValueN', '+Language=langCode']
```

prediction then is converted to follow the uncovered file pattern
```
langCode	wordForm	lemma	POS	Tag1=Value1|...|TagN=ValueN
```

#### embeddings
* word-level input embeddings
* character-level output embeddings
* learned
* initialized with random

#### data modification

In [0]:
class TrainDataModifyer:
    def modify_src_line(line):
        wf = line.split(' ')
        return ''.join(wf)


    def restore_src_line(line):
        wf = line
        return ' '.join([c for c in wf])


    def modify_tgt_line(line):
        return line
    

    def restore_tgt_line(line):
        return line




class NBestDataModifyer:
    
    def sent_to_baseline_compatible(line):
        line_pref = line.split("[")[0]+"["
        line_sent = line.split("[")[1].rstrip("]").strip('\'')
        return line_pref+ ', '.join('\''+c+'\'' for c in line_sent)+"]"
                       
    def hyp_to_baseline_compatible(line):
        return line


#### ml

In [0]:
ml(langs, tracks, train_params, TrainDataModifyer, NBestDataModifyer)

# sandbox